In [47]:
import numpy as np 
from numpy.random import choice
from tuner.dataset import ClassificationDataset
from tuner import load_data
from tuner import utils
import pickle
import chainer
import pandas as pd
import os

In [2]:
!ls models 

VGG.py	     birds_online.180110.hdf5	      birds_online.model.pkl
__init__.py  birds_online.20180125.hdf5
__pycache__  birds_online.20180125.model.pkl


In [9]:
class LeNet(chainer.Chain):
    def __init__(self, n_out=10):
        super(LeNet, self).__init__()
        
        initialW= I.HeNormal()
        with self.init_scope():
            self.conv0 = L.Convolution2D(None, 16, 1, 1, 0, initialW=initialW, nobias=True)
            
            self.bn11 = L.BatchNormalization(16)
            self.conv11 = L.Convolution2D(None, 32, 3, 1, 1, initialW=initialW, nobias=True)
            self.bn12 = L.BatchNormalization(32)
            self.conv12 = L.Convolution2D(None, 32, 3, 1, 1, initialW=initialW, nobias=True)
            
            self.bn21 = L.BatchNormalization(32)
            self.conv21 = L.Convolution2D(None, 64, 3, 1, 1, initialW=initialW, nobias=True)
            self.bn22 = L.BatchNormalization(64)
            self.conv22 = L.Convolution2D(None, 64, 3, 1, 1, initialW=initialW, nobias=True)
            
            self.conv_out = L.Convolution2D(None, n_out)
            self.fc = L.Linear(None, 20)
            self.l1 = L.Linear(None, 20)
            self.l2 = L.Linear(None, n_out)
    
    def __call__(self, x, u):
        h = x
        h = self.conv0(h)
        
        h = self.bn11(h)
        h = F.relu(h)
        h = self.conv11(h)
        h = self.bn12(h)
        h = F.relu(h)
        h = self.conv12(h)
        h = F.average_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.4)
        
        h = self.bn21(h)
        h = F.relu(h)
        h = self.conv21(h)
        h = self.bn22(h)
        h = F.relu(h)
        h = self.conv22(h)
        h = F.average_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.5)
        
        h = self.fc(h)
        #h = F.relu(h)
        h = F.dropout(h, ratio=0.5)
        h = F.concat((h, u), axis=1)
        h = F.dropout(h, ratio=0.8)
        h = self.l1(h)
        h = F.relu(h)
        h = F.dropout(h, ratio=0.6)
        h = self.l2(h)
        
        return h

In [10]:
with open('models/birds_online.20180125.model.pkl', 'rb') as f:
    m = pickle.load(f)

In [11]:
birds = ClassificationDataset('datasets/birds-dataset')

In [ ]:
val_data = '5a606243b037cb270b56c29c/

In [16]:
!ls standard_datasets/5a6062430

ls: 'standard_datasets/5a606243b037cb270' にアクセスできません: そのようなファイルやディレクトリはありません


In [17]:
val_data = ClassificationDataset('standard_datasets/5a606243b037cb270b56c29c/validation/')

In [25]:
df_num = pd.read_csv(os.path.join('datasets/birds-dataset', 'numeric_data.csv'))
df = pd.concat([val_data.df, df_num,], axis=1, join='inner')
u = df[['length','length']].values

In [96]:
df = load_data.df_fromdir_classed('standard_datasets/5a606243b037cb270b56c29c/validation/')

df_num = pd.read_csv(os.path.join('datasets/birds-dataset', 'numeric_data.csv'))

df = pd.concat([df, df_num,], axis=1, join='inner')

labels = sorted(list(set(df['label'])))
l2i = {l:i for i, l in enumerate(labels)}

def normalized_data(xs, us, ys):
    return (
        np.swapaxes(xs/255., 1, 3).astype(np.float32),
        utils.maxmin_normalize(us).astype(np.float32),
        ys.astype(np.int8))

x_val, y_val = load_data.load_fromdf(df, label2id=l2i, resize=128, rescale=1)

u_val = df[['length','weight']].values

xs, us, ys = normalized_data(x_val, u_val, y_val)

In [88]:
with open('models/birds_online.20180126.model.pkl', 'rb') as f:
    m = pickle.load(f)

In [84]:
import chainer.functions as F

In [91]:
out.data.shape

(66, 6)

In [93]:
xs, us, ys = normalized_data(x_val, u_val, y_val)
out = F.softmax(m.predictor(xs, us))

In [94]:
df_out = pd.DataFrame(out.data, columns=labels)

ValueError: Shape of passed values is (6, 66), indices imply (1, 66)

In [80]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
df_out.ix[0].plot.bar()

NameError: name 'df_out' is not defined

In [5]:
def undersampling_df(data_frame, sampling_size=None):
    df = data_frame
    labels = set(df['label'])

    sampling_size =\
        sampling_size if sampling_size else\
        min(df['label'].value_counts())
    sampling_idx = np.array([
        choice(df[df.label == label].index, sampling_size, replace=False) for label in labels
    ]).flatten()
    sampling_data = df.loc[sampling_idx].reset_index()
    return sampling_data

In [6]:
df = undersampling_df(birds.df_train)

In [7]:
df.label.value_counts()

yellow_finch     105
white_parrot     105
yellow_parrot    105
white_finch      105
red_parrot       105
red_finch        105
Name: label, dtype: int64

In [8]:
def oversampling_df(data_frame, sampling_size=None):
    df = data_frame
    labels = set(df['label'])

    sampling_size =\
        sampling_size if sampling_size else\
        max(df['label'].value_counts())

    def sampling_idx_each_label(label):
        idx = df[df.label == label].index
        return \
            choice(idx, sampling_size, replace=False) if sampling_size < len(idx) else\
            np.concatenate([idx, choice(idx, sampling_size - len(idx))])

    sampling_idx =\
        np.array([sampling_idx_each_label(label)
                  for label in labels]).flatten()
    sampling_data = df.loc[sampling_idx].reset_index()
    return sampling_data

In [9]:
df = oversampling_df(birds.df_train)

In [10]:
df.label.value_counts()

yellow_parrot    183
white_finch      183
red_parrot       183
red_finch        183
yellow_finch     183
white_parrot     183
Name: label, dtype: int64

In [12]:
load_data.load_fromdf(birds.df_val)

(array([[[[ 148.,  130.,   64.],
          [ 145.,  130.,   64.],
          [ 143.,  132.,   64.],
          ..., 
          [ 126.,  128.,   69.],
          [ 126.,  127.,   70.],
          [ 126.,  127.,   70.]],
 
         [[ 142.,  131.,   63.],
          [ 140.,  131.,   65.],
          [ 138.,  131.,   66.],
          ..., 
          [ 123.,  131.,   68.],
          [ 124.,  130.,   69.],
          [ 123.,  129.,   69.]],
 
         [[ 136.,  130.,   63.],
          [ 135.,  131.,   65.],
          [ 134.,  132.,   67.],
          ..., 
          [ 122.,  133.,   67.],
          [ 122.,  132.,   66.],
          [ 121.,  131.,   67.]],
 
         ..., 
         [[  40.,   40.,   48.],
          [  44.,   43.,   52.],
          [  45.,   44.,   52.],
          ..., 
          [  78.,   76.,   77.],
          [  59.,   58.,   59.],
          [  61.,   61.,   67.]],
 
         [[  66.,   63.,   66.],
          [  51.,   50.,   58.],
          [  50.,   49.,   60.],
          ..., 
  

In [15]:
birds.i2l

{0: 'red_finch',
 1: 'red_parrot',
 2: 'white_finch',
 3: 'white_parrot',
 4: 'yellow_finch',
 5: 'yellow_parrot'}

In [39]:
set(labels)

{'red_finch',
 'red_parrot',
 'white_finch',
 'white_parrot',
 'yellow_finch',
 'yellow_parrot'}